<a href="https://colab.research.google.com/github/gorogoro-uk/NLP-project/blob/master/Sherlock_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sherlock Holmes Text prediction project

In [1]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# import code
import os
from pathlib import Path
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import numpy as np
import tensorflow.keras.utils as ku
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras import regularizers

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
# define working directory
os.chdir('./drive/My Drive/sherlock')
print(f"listdir:    {os.listdir(os.getcwd())}")
STORY_FILE = './sh_short.txt'
print(f"STORY_FILE: {STORY_FILE}")

listdir:    ['sh_short.txt']
STORY_FILE: ./sh_short.txt


In [4]:
# import story text file
with open(STORY_FILE) as reader:
    raw_data = reader.read()   # raw_data is very long string

print(f"raw_data type:    {type(raw_data)}")
print(f"raw_data length:  {len(raw_data)}")
print(raw_data[:300])

raw_data type:    <class 'str'>
raw_data length:  46514

I. A SCANDAL IN BOHEMIA.
I.
To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particular


In [5]:
# remove chapter numbers [I. II. III. etc]
tx1 = raw_data.replace('II.','')
tx1 = tx1.replace('III.','')
tx1 = tx1.replace('IV.','')
tx1 = tx1.replace('V.','')
tx1 = tx1.replace('VI.','')
tx1 = tx1.replace('VII.','')
tx1 = tx1.replace('VIII.','')
tx1 = tx1.replace('IX.','')
tx1 = tx1.replace(' X.',' ')
tx1 = tx1.replace('XI.','')
tx1 = tx1.replace('XII.','')
tx1 = tx1.replace('\nI.','\n')
print(f"tx1:\n{tx1[:300]}")

# replace end of line with space: out=long string
# keep sentence ending punctuation [.?!] to allow sentence splitting below
# include other angled quote marks found in the text [”“‘’]
print(f"punctuation:  {string.punctuation}")
punc_string = "\"#$%&'()*+,-/:;<=>@[\]^_`{|}~'”“‘’—"
print(f"punc string:  {punc_string}")
tt = tx1.maketrans('', '', punc_string)
tx2 = tx1.translate(tt)
tx2 = tx2.replace('\n',' ')
print(f"tx2:\n{tx1[:300]}")

# split into sentences: out=list of strings, each one a sentence
tx3 = nltk.tokenize.sent_tokenize(tx2)
print(f"tx3:\n{tx3[:300]}")
print(f"type:    {type(tx3)}")
print(f"length:  {len(tx3)}")

# remove sentence ending punctuation [.?!]
# make lowercase
# remove white space at start/end
tx4 = [s.translate(s.maketrans('','','.?!')) for s in tx3]
tx4 = [s.lower() for s in tx4]
tx4 = [s.strip() for s in tx4]
print(f"tx4:\n{tx4[:300]}")

tx1:

 A SCANDAL IN BOHEMIA.

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,

punctuation:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
punc string:  "#$%&'()*+,-/:;<=>@[\]^_`{|}~'”“‘’—
tx2:

 A SCANDAL IN BOHEMIA.

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,

tx3:
['  A SCANDAL IN BOHEMIA.', 'To Sherlock Holmes she is always the woman.', 'I have seldom heard him mention her under any other name.', 'In his eyes she eclipses and predominates the whole of her sex.', 'It was not that he felt any emotion akin to love for Irene Adler.', 'All emotions 

In [6]:
# remove stopwords
# keep all stopwords (may activate in future)
tx5 = tx4

In [30]:
# tokenize words, create word_index
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tx5)
total_words = len(tokenizer.word_index) + 1
print(f"Total words: {total_words}")
print(f"tokenized text: {tokenizer.get_config()}")

# create numer sequences from word sentences
tx6 = tokenizer.texts_to_sequences(tx5)
print(f"tx6, tokenized sentences:\n{tx6[:300]}")

Total words: 1951
tokenized text: {'num_words': None, 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 'lower': True, 'split': ' ', 'char_level': False, 'oov_token': None, 'document_count': 669, 'word_counts': '{"a": 225, "scandal": 4, "in": 156, "bohemia": 8, "to": 245, "sherlock": 11, "holmes": 47, "she": 71, "is": 86, "always": 5, "the": 446, "woman": 11, "i": 259, "have": 62, "seldom": 3, "heard": 8, "him": 24, "mention": 1, "her": 48, "under": 8, "any": 4, "other": 12, "name": 6, "his": 105, "eyes": 9, "eclipses": 1, "and": 255, "predominates": 1, "whole": 4, "of": 236, "sex": 1, "it": 153, "was": 124, "not": 60, "that": 103, "he": 109, "felt": 2, "emotion": 2, "akin": 2, "love": 5, "for": 46, "irene": 13, "adler": 13, "all": 28, "emotions": 1, "one": 27, "particularly": 2, "were": 26, "abhorrent": 1, "cold": 2, "precise": 1, "but": 53, "admirably": 2, "balanced": 1, "mind": 6, "take": 7, "most": 11, "perfect": 1, "reasoning": 3, "observing": 2, "machine": 1, "world": 1, "has": 

In [31]:
# create n-grams of increasing length from each sentence sequence
tx7 = []
for sent_seq in tx6:
   for i in range(1, len(sent_seq)):
        n_gram_seq = sent_seq[:i+1]
        tx7.append(n_gram_seq)

print(f"tx7, n-gram sequences:\n{tx7[:300]}")

tx7, n-gram sequences:
[[6, 242], [6, 242, 7], [6, 242, 7, 133], [4, 94], [4, 94, 27], [4, 94, 27, 16], [4, 94, 27, 16, 14], [4, 94, 27, 16, 14, 197], [4, 94, 27, 16, 14, 197, 1], [4, 94, 27, 16, 14, 197, 1, 95], [2, 18], [2, 18, 320], [2, 18, 320, 134], [2, 18, 320, 134, 56], [2, 18, 320, 134, 56, 766], [2, 18, 320, 134, 56, 766, 26], [2, 18, 320, 134, 56, 766, 26, 135], [2, 18, 320, 134, 56, 766, 26, 135, 243], [2, 18, 320, 134, 56, 766, 26, 135, 243, 89], [2, 18, 320, 134, 56, 766, 26, 135, 243, 89, 171], [7, 12], [7, 12, 118], [7, 12, 118, 16], [7, 12, 118, 16, 767], [7, 12, 118, 16, 767, 3], [7, 12, 118, 16, 767, 3, 768], [7, 12, 118, 16, 767, 3, 768, 1], [7, 12, 118, 16, 767, 3, 768, 1, 244], [7, 12, 118, 16, 767, 3, 768, 1, 244, 5], [7, 12, 118, 16, 767, 3, 768, 1, 244, 5, 26], [7, 12, 118, 16, 767, 3, 768, 1, 244, 5, 26, 769], [8, 10], [8, 10, 21], [8, 10, 21, 13], [8, 10, 21, 13, 11], [8, 10, 21, 13, 11, 458], [8, 10, 21, 13, 11, 458, 243], [8, 10, 21, 13, 11, 458, 243, 459], 

In [32]:
# pre-pad sequences to common size
# convert to numpy array
max_seq_len = max([len(x) for x in tx7])
print(max_seq_len)
tx8 = np.array(pad_sequences(tx7, maxlen=max_seq_len, padding='pre'))
print(f"tx8, padded sequences:\n{tx8[:300]}")
print(tx8.dtype)

79
tx8, padded sequences:
[[  0   0   0 ...   0   6 242]
 [  0   0   0 ...   6 242   7]
 [  0   0   0 ... 242   7 133]
 ...
 [  0   0   0 ... 829  41   1]
 [  0   0   0 ...  41   1 334]
 [  0   0   0 ...   1 334   5]]
int32


In [33]:
# create predictors and label
features = tx8[:,:-1]
print(f"features:\n{features.shape}\n{features[:3,:]}")
labels_seq = np.expand_dims(tx8[:,-1], axis=1)                          # tokenizer encoding
print(f"labels_seq:\n{labels_seq.shape}\n{labels_seq}")
labels_hot = ku.to_categorical(labels_seq, num_classes=total_words)     # one hot encoding
print(f"labels_hot:\n{labels_hot.shape}\n{labels_hot}")

features:
(7853, 78)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   6]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   6 242]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   6 242   7]
 [  0   0 

In [34]:
# prepare model
# call back or learning rate, tensorboard visualisation
embedding_size = 100

sh_model = Sequential([
  Embedding(total_words, embedding_size, input_length=max_seq_len-1),                   # [batch, sentence(78), embedding(100)] 
  Bidirectional(LSTM(150, return_sequences = True)),                                    # [batch, 78, 300]
  Dropout(0.2),                                                                         # [batch, 78, 300]
  LSTM(100),                                                                            # [batch, 100]
  Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)),    # [batch, 975]
  Dense(total_words, activation='softmax')                                              # [batch, 1951]
])                                             

sh_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


In [35]:
sh_model_hist = sh_model.fit(features, labels_hot, epochs=5, verbose=2)

Epoch 1/5
246/246 - 85s - loss: 6.7017 - accuracy: 0.0516
Epoch 2/5
246/246 - 84s - loss: 6.0038 - accuracy: 0.0537
Epoch 3/5
246/246 - 84s - loss: 5.8123 - accuracy: 0.0632
Epoch 4/5
246/246 - 84s - loss: 5.6832 - accuracy: 0.0704
Epoch 5/5
246/246 - 84s - loss: 5.5781 - accuracy: 0.0781
